# 동요 만들기

## 사전 코드 정의

In [0]:
code2idx = {'c4':0, 'd4':1, 'e4':2, 'f4':3, 'g4':4, 'a4':5, 'b4':6,
            'c8':7, 'd8':8, 'e8':9, 'f8':10, 'g8':11, 'a8':12, 'b8':13}

idx2code = {0:'c4', 1:'d4', 2:'e4', 3:'f4', 4:'g4', 5:'a4', 6:'b4',
            7:'c8', 8:'d8', 9:'e8', 10:'f8', 11:'g8', 12:'a8', 13:'b8'}

# c(도) d e f g a(라) b(시) 4(4분음표) 8(8분음표) 
# 시퀀스 데이터 정의

seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4',
       'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4',
       'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4',
       'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']

In [0]:
import numpy as np
def seq2dataset(seq,window_size):
  dataset=[]
  for i in range(len(seq)-window_size): # training시, 0~49까지만 입력으로 들어가도록 선언해줌(윈도사이즈4니까)
    subset=seq[i:(i+window_size+1)]
    dataset.append([code2idx[item] for item in subset]) # g8 -> 11
    
    
  return np.array(dataset)
#   print(dataset)
#   print(len(seq)) #54

In [0]:
dataset=seq2dataset(seq,window_size=4)
print(dataset.shape)

(50, 5)


In [0]:
dataset

array([[11,  9,  2, 10,  8],
       [ 9,  2, 10,  8,  1],
       [ 2, 10,  8,  1,  7],
       [10,  8,  1,  7,  8],
       [ 8,  1,  7,  8,  9],
       [ 1,  7,  8,  9, 10],
       [ 7,  8,  9, 10, 11],
       [ 8,  9, 10, 11, 11],
       [ 9, 10, 11, 11,  4],
       [10, 11, 11,  4, 11],
       [11, 11,  4, 11,  9],
       [11,  4, 11,  9,  9],
       [ 4, 11,  9,  9,  9],
       [11,  9,  9,  9, 10],
       [ 9,  9,  9, 10,  8],
       [ 9,  9, 10,  8,  1],
       [ 9, 10,  8,  1,  7],
       [10,  8,  1,  7,  9],
       [ 8,  1,  7,  9, 11],
       [ 1,  7,  9, 11, 11],
       [ 7,  9, 11, 11,  9],
       [ 9, 11, 11,  9,  9],
       [11, 11,  9,  9,  2],
       [11,  9,  9,  2,  8],
       [ 9,  9,  2,  8,  8],
       [ 9,  2,  8,  8,  8],
       [ 2,  8,  8,  8,  8],
       [ 8,  8,  8,  8,  8],
       [ 8,  8,  8,  8,  9],
       [ 8,  8,  8,  9,  3],
       [ 8,  8,  9,  3,  9],
       [ 8,  9,  3,  9,  9],
       [ 9,  3,  9,  9,  9],
       [ 3,  9,  9,  9,  9],
       [ 9,  9

## 다층 퍼셉트론 모델

In [0]:
from keras.utils import np_utils
max_idx_value=13
xtrain=dataset[:,0:4]
ytrain=dataset[:,4]
xtrain=xtrain/float(max_idx_value) # 정규화
ytrain=np_utils.to_categorical(ytrain) # 원핫인코딩
ytrain.shape # (50, 12) 주어진 음 14개 중에서 12게 사용
one_hot_vec_size=ytrain.shape[1]

In [0]:
print("원 핫 벡터의 크기는 :", one_hot_vec_size)

원 핫 벡터의 크기는 : 12


In [0]:
import keras
from keras.models import *
from keras.layers import *
from keras.utils import *
model=Sequential()
model.add(Dense(128,input_dim=4,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(one_hot_vec_size,activation='softmax'))

## 학습 알고리즘 설정

In [0]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [0]:
class LossHistory(keras.callbacks.Callback):
  def init(self): # 현재만들어지는 객체를 의미함
    self.losses=[]
    
  def on_epoch_end(self, batch, log={}): # 시스템에 의해 자동으로 불러지는 콜백함수임 # 에폭이 끝났을 때라는 의미
    self.losses.append(log.get('loss'))    

In [0]:
history=LossHistory()
history.init()

In [0]:
num_epochs=1000 #성능이 안좋을 것임...

model.fit(xtrain, ytrain, epochs=num_epochs, batch_size=1, verbose=2, callbacks=[history])

In [0]:
scores=model.evaluate(xtrain,ytrain,batch_size=1)
scores
print("%.2f%%" % (scores[1]*100))

50/50 [==============================] - 0s 863us/step
92.00%


In [0]:
seq_out=['g8', 'e8', 'e4', 'f8']
pred_out=model.predict(xtrain,batch_size=1)
np.argmax(pred_out,1) #해당 인덱스 

array([ 8,  1,  7,  9,  9, 10, 11, 11,  4, 11,  9,  9,  9, 10,  4,  1,  7,
        9, 11, 11,  9,  9,  2,  8,  8,  8,  8,  9,  9,  3,  9,  9,  9,  9,
       10, 10,  4, 11,  9,  2, 10,  8,  1,  7,  9, 11, 11,  9,  9,  2])

In [0]:
for i in range(50): # 예측해야 할 갯수 50개
  idx=np.argmax(pred_out[i])
  seq_out.append(idx2code[idx])
#   print(idx)
seq_out